# Week 2: More on Regression: Examples, Basis Functions, and Kernels

## Lecture 4 (Wed, Jan 20)

Continuing from our ordinary least squares regression model from last week, we will first work on a somewhat extensive example with a real dataset. It will be a pretty realistic problem because the data is not quite pristine and will require some preprocessing before it can be modeled well.

To facilitate that, we import some libraries and functions.

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

### Example: Multivariate Regression for Air Quality 

For this example, we will use the Beijing Multi-Site Air-Quality Data Data Set dataset$^1$ available from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data). It is a hourly data set considers 6 main air pollutants and 6 relevant meteorological variables at multiple sites in Beijing.

[1] Zhang, S., Guo, B., Dong, A., He, J., Xu, Z. and Chen, S.X. (2017) Cautionary Tales on Air-Quality Improvement in Beijing. *Proceedings of the Royal Society A*, Volume 473, No. 2205. https://doi.org/10.1098/rspa.2017.0457

The dataset includes a number of variables associated with time, weather, and location: the year, month, day, hour, temperature, pressure, dew point, precipitation, wind direction, wind speed, and station name.

The dataset also includes air pollutant levels for: fine inhalable particulate matter with diameter $\leq 2.5\,\mu$m (PM$_{2.5}$), inhalable particles with diameter $\leq 10\, \mu$m (PM$_{10}$), sulfur dioxide (SO$_2$), nitrogen dioxide (NO$_2$), carbon monoxide (CO), and ozone (O$_3$).

We will try to predict these pollution levels based the time, weather, and location variables.

This dataset is not as neat as the US high school graduation data: there are numerical variables, but there are a few new wrinkles:

* Some variables are text
* There is missing numbers in the dataset
* There is a variable that simply indexes the data

All of these issues would prevent us from using ordinary least squares, so we need to solve these issues. In most applied problems, there are similar issues that must be managed before you can do much machine learning.

#### Cleaning/Preprocessing the Data

First, we must **clean** the data (manipulate it into a data matrix we can use for machine learning). The data is stored in 12 separate comma-separated value (CSV) files, so we will use the `glob` library to iterate through the files and use the `pandas` library to store each as a dataframe and then concatenate them into one big dataframe.

In [15]:
import glob

# import all the files in data/PRSA
dataFiles = glob.glob('data/PRSA/*')

# empty list to store the data
data = []

# iterate through files, read files
for file in dataFiles:
    # convert file to dataframe and add it to the list
    data.append(pd.read_csv(file, sep = ','))
    
# concatenate all the dataframes into one
data = pd.concat(data)

# display the data
data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,NW,2.4,Wanshouxigong
35060,35061,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Wanshouxigong
35061,35062,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,NW,1.1,Wanshouxigong
35062,35063,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Wanshouxigong


Next, we can use pandas to drop unimportant variables:

* Drop the '**No**' column with `drop()` since it just stores an index that has no physical significance.
* Drop rows with empty values with `dropna()`.

In [16]:
# drop the 'No' column
data = data.drop(columns = ['No'])

# drop rows with missing data
data = data.dropna()

# display the data
data

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,NW,2.4,Wanshouxigong
35060,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Wanshouxigong
35061,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,NW,1.1,Wanshouxigong
35062,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Wanshouxigong


#### Converting a Categorical Variable to One-Hot

Note the number of datapoints went from 420768 to 382168, a loss of about 9\% of the data, due to some data being incomplete. Simply dropping datapoints can sometimes bias the model, but this is a relatively small amount of data, so it should not be a big problem.

The next problem we have is that the **station** variable is not numerical but is rather categorical, representing the site where the datapoint was measured. This does not work with ordinary least squares. We could delete them from the dataset as well, but these sites are in different parts of the city which may experience different pollution patterns, so this information may help the model make predictions.

One way to deal with categorical variables in machine learning problems is to convert them to **one-hot vectors** which are standard basis vectors of $\mathbb{R}^m$ where $m$ is the number of categories. In other words, they are made up of all 0s except for one 1, representing the one category in the datapoint.

Luckily, `Pandas` has a `get_dummies()` function, which does this conversion for us!

In [17]:
# convert the 'station' column to binary variables
data = pd.get_dummies(data, columns = ['station'])

# display the data
data

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,...,station_Dingling,station_Dongsi,station_Guanyuan,station_Gucheng,station_Huairou,station_Nongzhanguan,station_Shunyi,station_Tiantan,station_Wanliu,station_Wanshouxigong
0,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,...,0,0,0,0,0,0,0,0,0,0
1,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,...,0,0,0,0,0,0,0,0,0,0
2,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,...,0,0,0,0,0,0,0,0,0,0
3,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,...,0,0,0,0,0,0,0,0,0,0
4,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,...,0,0,0,0,0,0,0,0,0,1
35060,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,...,0,0,0,0,0,0,0,0,0,1
35061,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,...,0,0,0,0,0,0,0,0,0,1
35062,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,...,0,0,0,0,0,0,0,0,0,1


Note the station variable has been replaced with a binary variable for each station. In all, we lost 1 column (**station**) and gained 12 more, **station_(station name)**, resulting in 28 total columns.

#### Wind Direction

The last problem we see is the wind direction column:

In [18]:
data['wd']

0        NNW
1          N
2        NNW
3         NW
4          N
        ... 
35059     NW
35060    WNW
35061     NW
35062    NNW
35063    NNE
Name: wd, Length: 382168, dtype: object

These values in the **wd** column represent the wind speed. So, why not simply replace them with one-hot vectors like the **station** column?

We could, but this obscures some information in the data. These are stored as categorical variables, but they correspond to angles. If we used one-hot vectors, we may lose that physical structure.

Another option is to simply convert them to angles and store them in radians as $\theta=0$, $\frac{\pi}{8}$, $\frac{2\pi}{8}$, ..., $\frac{15\pi}{8}$. (In reality, different angles are possible, but the original data was rouded to these values.) This leads to another problem: here, it will appear to the algorithm that, for example, the difference between an angles of $0$ and $\frac{15\pi}{8}$ will be much greater than the difference between angles of $0$ and $\pi$, which is not quite right in this context.

A better solution is to store the wind direction as a vector on the unit circle as $(\cos(\theta), \sin(\theta))$. So, let's write a simple function that converts radian measures to this vector.

In [19]:
# convert an angle to a list of unit circle coordinates
def unitCircle(angle):
    return [np.cos(angle), np.sin(angle)]

Then, let's make a dictionary that maps each direction to the appropriate angle

In [20]:
# list all wind directions, along the unit circle
directions = ['E', 'ENE', 'NE', 'NNE', 'N', 'NNW', 'NW', 'WNW', 'W', 'WSW', 'SW', 'SSW', 'S', 'SSE', 'SE', 'ESE']

# make a dictionary associating each direction with coordinates on the unit circle 
directionDict = {direction : unitCircle(i*np.pi/8) for (i, direction) in enumerate(directions)}

# create a dataframe from the dictionary
directionDf = pd.DataFrame.from_dict(directionDict, orient = 'index', columns = ['unitX', 'unitY'])

# display the dataframe
directionDf

,unitX,unitY
E,1.000000e+00,0.000000e+00
ENE,9.238795e-01,3.826834e-01
NE,7.071068e-01,7.071068e-01
NNE,3.826834e-01,9.238795e-01
N,6.123234e-17,1.000000e+00
NNW,-3.826834e-01,9.238795e-01
NW,-7.071068e-01,7.071068e-01
WNW,-9.238795e-01,3.826834e-01
W,-1.000000e+00,1.224647e-16
WSW,-9.238795e-01,-3.826834e-01


Our goal will be to add **unitX** and **unitY** columns to our `data` dataframe, map the existing **wd** value to the appropriate $x$ and $y$ coordinates, and delete the **wd** column.

`pandas` has the `join` function to take the dataframe we just created to do just this mapping.

In [21]:
# join the direction dataframe with the data, mapping directions to unit circle coordinates
data = data.join(directionDf, on = 'wd')

# drop the wind direction column
data = data.drop(columns = ['wd'])

# display the data
data

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,...,station_Guanyuan,station_Gucheng,station_Huairou,station_Nongzhanguan,station_Shunyi,station_Tiantan,station_Wanliu,station_Wanshouxigong,unitX,unitY
0,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,...,0,0,0,0,0,0,0,0,-3.826834e-01,0.923880
1,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,...,0,0,0,0,0,0,0,0,6.123234e-17,1.000000
2,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,...,0,0,0,0,0,0,0,0,-3.826834e-01,0.923880
3,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,...,0,0,0,0,0,0,0,0,-7.071068e-01,0.707107
4,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,...,0,0,0,0,0,0,0,0,6.123234e-17,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,...,0,0,0,0,0,0,0,1,-7.071068e-01,0.707107
35060,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,...,0,0,0,0,0,0,0,1,-9.238795e-01,0.382683
35061,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,...,0,0,0,0,0,0,0,1,-7.071068e-01,0.707107
35062,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,...,0,0,0,0,0,0,0,1,-3.826834e-01,0.923880


Finally, our data is totally numerical, so we can now fit a regression model!

#### Train and Test Sets

We convert the pollutant columns to a `NumPy` array by applying the `to_numpy()` function to `data`, selecting just the pollutant columns. These are our responses, or $y$ variables in the regression problem, `dataY`. We take the opposite columns to be the data matrix, `dataX`.

Then, we use the `train_test_split` function to randomly assign 75\% of the data to the training set and 25\% to the test set.

In [22]:
pollutants = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

# pollutants are the responses
dataY = data[pollutants].to_numpy()

# all data except pollutants are predictors
dataX = data.drop(columns = pollutants).to_numpy()

# split the dataset and labels to 75% training set and 25% test set
trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size = 0.25, random_state = 1)

Let's print the dimensions of our newly created training and test data to see if it makes sense.

In [23]:
print('Training set dimensions')
print(trainX.shape)
print(trainY.shape)

print('\nTest set dimensions')
print(testX.shape)
print(testY.shape)

Training set dimensions
(286626, 23)
(286626, 6)

Test set dimensions
(95542, 23)
(95542, 6)


#### Fitting Separate Models for Each Response Variable

Note that the training set will be quite large (286626 x 286626), which will be difficult with our simple matrix multiplication in the `OrdinaryLeastSquares` class we write, as it is likely to lead to overflows, so let's use the optimized `LinearRegression` class built into the popular `scikit-learn` library and predict each pollutant separately. If you are interested in linear algebra, it is good to know it uses singular value decomposition (SVD).

In [24]:
# import the linear regression model
from sklearn.linear_model import LinearRegression

# instantiate an OLS model
model = LinearRegression()

for i in range(trainY.shape[1]):
    # choose the pollutant
    print('\n==================== Modeling', pollutants[i], 'pollution =====================')

    # fit the model to the training data (find the beta parameters)
    model.fit(trainX, trainY[:, i])

    # return the predicted outputs for the datapoints in the training set
    trainPredictions = model.predict(trainX)

    # print the coefficient of determination r^2
    print('The r^2 score is', model.score(trainX, trainY[:, i]))

    # print quality metrics
    print('The mean absolute error on the training set is', mean_absolute_error(trainY[:, i], trainPredictions))
    
    # print the beta values
    #print('The beta values are', model.beta)
    print('The beta values are', np.round(model.coef_, 2))

    # return the predicted outputs for the datapoints in the test set
    predictions = model.predict(testX)

    # print quality metrics
    print('The mean absolute error on the test set is', mean_absolute_error(testY[:, i], predictions))


==================== Modeling PM2.5 pollution =====================
The r^2 score is 0.23510056982521432
The mean absolute error on the training set is 50.4452441776565
The beta values are [ -1.63  -1.33   0.     1.4   -6.03  -1.38   3.78  -4.81  -2.93   1.
  -5.7  -10.36   8.58   1.85   3.08 -18.2    6.82   2.71   3.83   0.57
   5.83   8.89 -17.08]
The mean absolute error on the test set is 50.261496050071955

==================== Modeling PM10 pollution =====================
The r^2 score is 0.16088318842493265
The mean absolute error on the training set is 60.58794671634826
The beta values are [ -2.95  -1.23   0.3    1.71  -5.89  -2.52   2.82  -5.94  -1.11   5.08
 -11.56 -21.97  11.65   4.52  11.63 -24.54   7.82  -0.21   4.36   4.07
   9.15   8.82 -19.96]
The mean absolute error on the test set is 60.53901994023608

==================== Modeling SO2 pollution =====================
The r^2 score is 0.26542293236601733
The mean absolute error on the training set is 11.999224649937082

#### Fitting a Multivariate Regression Model

The `OrdinaryLeastSquares` class we wrote was for multiple regression (multiple inputs) but not multivariate regression (multiple outputs). We will again use the `LinearRegression` class from `scikit-learn`.

In [25]:
# instantiate an OLS model
model = LinearRegression()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# print the coefficient of determination r^2
print('The r^2 score is', model.score(trainX, trainY))

# print quality metrics
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

# print the beta values
betas = [np.round(row, 2) for row in model.coef_]
for i in range(6):
    print('\nThe beta values for predicting', pollutants[i], 'are\n', betas[i])

# print quality metrics
print('\nThe mean absolute error on the test set is', mean_absolute_error(testY, predictions))

The r^2 score is 0.3124097213393054
The mean absolute error on the training set is 138.44233663696912

The beta values for predicting PM2.5 are
 [ -1.63  -1.33   0.     1.4   -6.03  -1.38   3.78  -4.81  -2.93   1.
  -5.7  -10.36   8.58   1.85   3.08 -18.2    6.82   2.71   3.83   0.57
   5.83   8.89 -17.08]

The beta values for predicting PM10 are
 [ -2.95  -1.23   0.3    1.71  -5.89  -2.52   2.82  -5.94  -1.11   5.08
 -11.56 -21.97  11.65   4.52  11.63 -24.54   7.82  -0.21   4.36   4.07
   9.15   8.82 -19.96]

The beta values for predicting SO2 are
 [-4.67 -1.28 -0.    0.23 -0.78 -0.2  -0.09 -0.51 -1.76  1.68 -0.34 -3.47
  2.1   1.71 -1.13 -4.76  2.93 -1.64 -1.3   2.66  1.55  2.28 -5.13]

The beta values for predicting NO2 are
 [ -1.64   0.15   0.06   0.63  -2.28  -0.69   0.77  -2.17  -7.41   8.64
  -5.72 -23.06   5.69   7.18   1.95 -22.63   9.45  -4.22   4.5   12.55
   5.67   3.33  -4.44]

The beta values for predicting CO are
 [   1.5    10.29   -2.29   17.04  -95.96  -22.61   42.01 

As you might note, there is actually no difference between the models in the last two code blocks. The parameters are the same for each pollutant. The only difference is that we have wholistic measures of quality, which are simply averages of the results from above.

In a sense, the linear regression models are just stacked on one another. It turns out, this is a common situation in machine learning. In fact, neural networks (deep learning) that solve regression problems can be considered as stacked models. The difference is that neural networks allows the responses variable to *share* parameters--so all response variables would be influenced by all of the parameters in the model.

### Kernel Regression